# Записи телеметрии


|               |    |
|:--------------|:---|
| Группа        | APT29 |
| Описание      | Группа угроз APT29 оперирует как минимум с 2008 года, в 2015 году провела одну из самых известных своих атак. Её содержание ниже. |
| Авторы         | [Воркшоп МЦА](http://mca.nsu.ru/projects/) |
    

## Импортируем библиотеки

In [ ]:
from pyspark.sql import SparkSession

## Запускаем сессию Spark

In [ ]:
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.caseSensitive", "true")

## Распаковываем логи атаки

In [ ]:
!wget https://github.com/hunters-forge/mordor/raw/master/datasets/large/apt29/day1/apt29_evals_day1_manual.zip

In [ ]:
!unzip apt29_evals_day1_manual.zip

## Импортируем логи атаки

In [ ]:
df_day1_host = spark.read.json('apt29_evals_day1_manual_2020-05-01225525.json')

## Создаём временный SQL View

In [ ]:
df_day1_host.createTempView('apt29Host')

## У таблицы apt29 множество столбцов, их можно посмотреть

In [ ]:
df = spark.sql(
'''
DESCRIBE apt29Host
'''
)
df.show(10,truncate = False, vertical = False)

## Нападающий - шаги обнаружения

## 1.A.1. User Execution
**Procedure:** Пользователь исполнил rcs.3aka3.doc

**Criteria:** Процесс rcs.3aka3.doc запущен процессом explorer.exe


### Как правило, данные лежат в логах sysmon. Про поля sysmon можно [почитать здесь](https://docs.microsoft.com/en-us/sysinternals/downloads/sysmon).

In [ ]:
df = spark.sql(
'''
SELECT Message
FROM apt29Host
WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
    AND EventID = 1
    AND LOWER(ParentImage) LIKE "%explorer.exe"
    AND LOWER(Image) LIKE "%3aka3%"

'''
)
df.show(100,truncate = False, vertical = True)

### Security-логи Windows немного не такие, как логи sysmon, иногда информация только в одном из этих логов. См [Sysmon Config.xml recommendations compared to Windows logs](https://static1.squarespace.com/static/552092d5e4b0661088167e5c/t/5d5588b51fd81f0001471db4/1565886646582/Windows+Sysmon+Logging+Cheat+Sheet_Aug_2019.pdf)

In [ ]:
df = spark.sql(
'''
SELECT Message
FROM apt29Host
WHERE LOWER(Channel) = "security"
    AND EventID = 4688
    AND LOWER(ParentProcessName) LIKE "%explorer.exe"
    AND LOWER(NewProcessName) LIKE "%3aka3%"

'''
)
df.show(100,truncate = False, vertical = True)

## 1.A.2. Masquerading
**Procedure:** Использован символ unicode right-to-left override (RTLO) для обфусцирования имени файла rcs.3aka3.doc (изначально cod.3aka.scr)

**Criteria:** Пристутствие right-to-left override character (U+202E) в процессе rcs.3aka.doc ИЛИ в исходном имени файла (cod.3aka.scr)


## Здесь ещё множество тактик, техник и процедур...

## 9.C.3. File Deletion
**Procedure:** Удалён working.zip (из каталога AppData) на диске используя SDelete

**Criteria:** sdelete64.exe удалил файл \AppData\Roaming\working.zip


In [ ]:
df = spark.sql(
'''
SELECT Message
FROM apt29Host j
INNER JOIN (
    SELECT h.ProcessGuid
    FROM apt29Host h
    INNER JOIN (
        SELECT f.ProcessGuid
        FROM apt29Host f
        INNER JOIN (
          SELECT d.ProcessGuid
          FROM apt29Host d
          INNER JOIN (
              SELECT b.ProcessGuid
              FROM apt29Host b
              INNER JOIN (
                SELECT ProcessGuid
                FROM apt29Host
                WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
                    AND EventID = 1
                    AND ParentImage LIKE '%services.exe'
              ) a
              ON b.ParentProcessGuid = a.ProcessGuid
              WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
                AND Image LIKE '%python.exe'
          ) c
          ON d.ParentProcessGuid = c.ProcessGuid
          WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
              AND EventID = 1
        ) e
        ON f.ParentProcessGuid = e.ProcessGuid
        WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
          AND EventID = 1
          AND Image LIKE '%cmd.exe'
    ) g
    ON h.ParentProcessGuid = g.ProcessGuid
    WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
        AND h.EventID = 1
) i
ON j.ProcessGuid = i.ProcessGuid
WHERE j.Channel = "Microsoft-Windows-Sysmon/Operational"
    AND j.EventID = 23

'''
)
df.show(100,truncate = False, vertical = True)

## 9.C.4. File Deletion
**Procedure:** SDelete удалён с диска, использована соманда del cmd.exe

**Criteria:** cmd.exe удалил файл sdelete64.exe


In [ ]:
df = spark.sql(
'''
SELECT h.Message
FROM apt29Host h
INNER JOIN (
    SELECT f.ProcessGuid
    FROM apt29Host f
    INNER JOIN (
      SELECT d.ProcessGuid
      FROM apt29Host d
      INNER JOIN (
          SELECT b.ProcessGuid
          FROM apt29Host b
          INNER JOIN (
            SELECT ProcessGuid
            FROM apt29Host
            WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
                AND EventID = 1
                AND ParentImage LIKE '%services.exe'
          ) a
          ON b.ParentProcessGuid = a.ProcessGuid
          WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
            AND Image LIKE '%python.exe'
      ) c
      ON d.ParentProcessGuid = c.ProcessGuid
      WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
          AND EventID = 1
    ) e
    ON f.ParentProcessGuid = e.ProcessGuid
    WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
      AND EventID = 1
      AND Image LIKE '%cmd.exe'
) g
ON h.ProcessGuid = g.ProcessGuid
WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
    AND h.EventID = 23

'''
)
df.show(100,truncate = False, vertical = True)